In [103]:
from llama_index.core.ingestion import IngestionPipeline
from llama_index.llms.deepseek import DeepSeek
import nest_asyncio

nest_asyncio.apply()

In [61]:
from llama_index.core import Document

# 模拟的 markdown 内容
markdown_text = """
马歇尔·卢森堡博士发现了一种沟通方式，依照它来谈话和聆听，能使人们情意相通，和谐相处，这就是“非暴力沟通”。
做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。不过如果稍微留意一下现实生活中的谈话方式，并且用心体会各种谈话方式给我们的不同感受，我们一定会发现，有些话确实伤人！言语上的指责、嘲讽、否定、说教以及任意打断、拒不回应、随意出口的评价和结论给我们带来的情感和精神上的创伤甚至比肉体的伤害更加令人痛苦。这些无心或有意的语言暴力让人与人变得冷漠、隔膜、敌视。
非暴力沟通能够：
● 疗愈内心深处的隐秘伤痛；
● 超越个人心智和情感的局限性；
● 突破那些引发愤怒、沮丧、焦虑等负面情绪的思维方式；
● 用不带伤害的方式化解人际间的冲突；
● 学会建立和谐的生命体验。
图书在版编目(CIP)数据
非暴力沟通／（美）马歇尔·卢森堡（Marshall B.Rosenberg）著；刘轶译.-2版（修订本）-北京：华夏出版社有限公司，2021.5
书名原文：Nonviolent Communication

# 序章

**作者:** 马歇尔·卢森堡  
**出版社:** 华夏出版社  
**出版时间:** 2021年
## 测试1
### 测试
---

非暴力沟通是一种语言方式，也是一种生活哲学。
它通过观察、感受、需要与请求四个步骤，让人们之间的沟通更有同理心、更具建设性。

# 第一章
XXXX
asdf
## 社保值长
nihao aasdasda
---
"""

# 创建 LlamaIndex 的 Document 对象
doc = Document(
    text=markdown_text,
)

# 打印确认
print("✅ Document 内容预览：")
print(doc.text[:100])  # 前100个字符
print("\n📎 Metadata:", doc.metadata)

✅ Document 内容预览：

马歇尔·卢森堡博士发现了一种沟通方式，依照它来谈话和聆听，能使人们情意相通，和谐相处，这就是“非暴力沟通”。
做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。不过如果稍微留意一下现

📎 Metadata: {}


In [85]:
from readai.components.epub2md_loader import get_nodes_from_string

md_nodes = get_nodes_from_string(text=markdown_text, metadata={})

该文档总共分割为 3 个节点


In [98]:
DEFAULT_SUMMARY_EXTRACT_TEMPLATE_ZH = """
以下是章节内容：
{context_str}

请总结本章节的核心主题和涉及的关键人物或实体。

摘要："""

In [ ]:
import json
import os

from dotenv import load_dotenv
from llama_index.core.extractors import SummaryExtractor

load_dotenv()
API_KEY = os.getenv("DEEPSEEK_API_KEY")
MODEL = os.getenv("DEEPSEEK_MODEL")

# 使用LLM创建摘要提取器
llm = DeepSeek(api_key=API_KEY, model=MODEL)
summary_extractor = SummaryExtractor(
    llm=llm,
    summaries=["self"],
    prompt_template=DEFAULT_SUMMARY_EXTRACT_TEMPLATE_ZH,
)
pipeline = IngestionPipeline(transformations=[summary_extractor])

# 处理节点，提取元数据,需要过滤非章节的节点
processed_nodes = pipeline.run(nodes=md_nodes, in_place=True, show_progress=True)


In [107]:
print(len(processed_nodes))
for node in processed_nodes:
    print(node.metadata)

3
{'section_summary': '**核心主题**：  \n本章节系统阐述了**非暴力沟通（NVC）**的理论与实践，强调通过特定的语言和倾听方式（如避免指责、评判等“暴力性”沟通）促进情感联结、化解冲突，并实现个人与社会的和谐关系。其核心价值包括疗愈情感创伤、突破负面情绪局限，以及构建非对抗性的人际互动模式。\n\n**关键人物与实体**：  \n1. **马歇尔·卢森堡博士**（Dr. Marshall Rosenberg）：NVC理论创始人，著作《非暴力沟通》为理论来源（2021年修订版，刘轶译）。  \n2. **语言暴力**：涵盖指责、嘲讽、否定、说教等伤害性沟通行为，是NVC旨在消除的对象。  \n3. **情感疗愈与冲突解决**：NVC的核心应用领域，聚焦于转化负面情绪（如愤怒、焦虑）和改善人际关系。  \n\n**总结**：章节揭示了日常沟通中的隐性暴力及其危害，同时提出NVC作为变革性工具，以实现深层共情与社会和谐。'}
{'Header1': '序章', 'section_summary': '**核心主题**：  \n本章节（序章）阐述了**非暴力沟通（NVC）**的双重性质，既是一种促进**同理心**和**建设性沟通**的**语言方法**，也是一种**生活哲学**。其核心框架包含四个步骤：**观察、感受、需要、请求**，旨在改善人际互动。\n\n**关键人物与实体**：  \n- **作者**：马歇尔·卢森堡（Marshall Rosenberg）  \n- **出版社**：华夏出版社（Huaxia Publishing House）  \n- **出版时间**：2021年  \n\n**关键术语**：非暴力沟通（NVC）、观察、感受、需要、请求、同理心。  \n\n（注：摘要保留了原文的结构与关键信息，精简了部分重复表述，确保内容清晰紧凑。）'}
{'Header1': '第一章', 'section_summary': '### 核心主题  \n- **章节定位**：标题为“第一章”，可能是文档的引言或框架性章节，但内容不完整，包含占位文本（如“XXXX”“asdf”）和非正式用语（如“nihao aasdasda”），表明其为未完成的草稿或模板。  \n- **内容特征**：无实质性信息，格式混杂（正式标题与非正式文本），需

In [110]:
book_id = "test_id"
json_file_name = f"chapter_summary_{book_id}.json"
# 按照标题，摘要组织成字典
chapter_dict = {}
for cid, node in enumerate(processed_nodes):
    if "Header1" not in node.metadata:
        continue
    summary = node.metadata["section_summary"]
    title = node.metadata["Header1"]
    chapter_dict[cid] = {"summary": summary, "title": title}
with open(json_file_name, "w") as f:
    json.dump(chapter_dict, f, ensure_ascii=False, indent=4)

In [111]:
# 检查每个节点都有哪些metadata字段
for node in processed_nodes:
    metadata = node.metadata
    print(metadata.keys())

dict_keys(['section_summary'])
dict_keys(['Header1', 'section_summary'])
dict_keys(['Header1', 'section_summary'])


In [16]:
from llama_index.core import Document
from readai.components.node_parsers import process_markdown_document

# 读取测试文件
with open("./test.md", encoding="utf-8") as f:
    markdown_content = f.read()

# 创建LlamaIndex的Document对象
document = Document(
    text=markdown_content, metadata={"author": "me", "date": "2025-01-01"}
)

nodes = process_markdown_document(
    document,
    min_chapter_count=8,  # 设置最小章节数
    max_chunk_size=100,
    strip_headers=True,  # 保留标题
)

# 打印结果
print(f"文档被分割为 {len(nodes)} 个节点")
for i, node in enumerate(nodes):
    print(f"\n节点 {i + 1}:")
    print(f"元数据: {node.metadata}")
    print(f"内容前100字符: {node.text[:100]}...")

文档被分割为 20 个节点

节点 1:
元数据: {'author': 'me', 'date': '2025-01-01', 'Header1': '创建一个包含不同标题级别和不同长度内容的测试Markdown文档', 'chunk_id': 0}
内容前100字符: test_markdown = """...

节点 2:
元数据: {'author': 'me', 'date': '2025-01-01', 'Header1': '第一章 绪论', 'chunk_id': 1}
内容前100字符: 这是第一章的内容，作为引言，内容比较简短。
非暴力沟通的基本概念将在此处简单介绍。...

节点 3:
元数据: {'author': 'me', 'date': '2025-01-01', 'Header1': '第一章 绪论', 'Header2': '1.1 研究背景', 'chunk_id': 2}
内容前100字符: 本小节讨论研究背景。这是一个二级标题的小节。...

节点 4:
元数据: {'author': 'me', 'date': '2025-01-01', 'Header1': '第二章 基本原理', 'sub_chunk': 1, 'chunk_id': 3}
内容前100字符: 这是一个非常长的章节，目的是测试解析器处理长内容的能力。
在非暴力沟通中，我们关注四个要素：观察、感受、需要和请求。这些要素构成了非暴力沟通的基本模型。...

节点 5:
元数据: {'author': 'me', 'date': '2025-01-01', 'Header1': '第二章 基本原理', 'sub_chunk': 2, 'chunk_id': 4}
内容前100字符: 观察是指清晰地表达我们观察到的事实，而不掺杂评价或判断。例如，"当我看到你..." 而不是 "你总是..."。
感受是指表达我们的情感状态，如高兴、悲伤、恐惧、愤怒等。表达感受需要我们区分真正的感受和...

节点 6:
元数据: {'author': 'me', 'date': '2025-01-01', 'Header1': '第二章 基本原理', 'sub_chunk': 3, 'chunk_id': 5}
内容前100字符: 需要是指表达我们普遍的人类需求，如安全、理解、